In [ ]:
# @title Установка
%cd /content/
!git clone https://github.com/noblebarkrr/mvsepless -b gamma /content/mvsepless-gamma
%cd /content/mvsepless-gamma
!pip install uv
!uv pip install --no-cache-dir -qq -r requirements.txt

In [ ]:
from pyngrok import ngrok

%cd /content/mvsepless-gamma
#@title Запуск интерфейса
#@markdown ### Интерфейс
#@markdown ---
#@markdown * Язык
language = "ru" # @param ["ru","en"]
#@markdown * Тема
theme = "gamma" # @param ["gamma","beta","alpha","mvsep","vbachgen"]
#@markdown * Поддержка плагинов
add_plugins = True # @param {type:"boolean"}
#@markdown ---
#@markdown ### Общий доступ
port = 7860
#@markdown * Способ поделится приложением
sharing_method = "ngrok" # @param ["gradio","ngrok"]
#@markdown * Токен для ngrok *(где взять его - https://dashboard.ngrok.com/get-started/your-authtoken)*
ngrok_token = "" # @param {"type":"string"}

if sharing_method == "ngrok":
    try:
        ngrok.set_auth_token(ngrok_token)
        ngrok.kill()
        tunnel = ngrok.connect(port)
        print(f"Публичная ссылка: {tunnel.public_url}")
    except KeyboardInterrupt:
        ngrok.kill()

if sharing_method == "localtunnel":
    pass

!python app.py {'--share' if sharing_method == "gradio" else ""} --port {port} --theme {theme} {'' if not add_plugins else '--plugins'} --lang {language}







In [ ]:

#@title Разделение через командную строку
#@markdown ### Входные данные
#@markdown * Путь к входной папке/файлу:
input_path = "" # @param {type:"string"}
#@markdown ---
#@markdown ### Выбор модели
#@markdown * Тип модели:
model_type = "bs_roformer" # @param ["mel_band_roformer", "bs_roformer", "mdx23c", "scnet", "htdemucs", "bandit", "bandit_v2", "vr", "mdx"]
#@markdown * Название модели:
model_name = "BS-Roformer_SW" # @param {"type":"string"}
#@markdown ---
#@markdown ### Настройки раздления
# @markdown * Степень подавления для VR моделей
vr_arch_aggressive = 5 # @param {"type":"slider","min":1,"max":100,"step":1}
# @markdown * Извлечь  инструментал:
instrumental = True # @param {type:"boolean"}
#@markdown ---
#@markdown ### Выходные данные
#@markdown * Формат:
output_format = "mp3" # @param ["mp3", "wav", "flac", "ogg", "opus", "m4a", "aac", "aiff"]
# @markdown * Битрейт
bitrate = 320 # @param {"type":"slider","min":32,"max":320,"step":1}
# @markdown * Выбрать выходные стемы(через пробел, например ("vocal"  "instrumental")):
stems_to_extract = "" # @param {type:"string"}
#@markdown * Шаблон именования выходных файлов:
output_template = "NAME (STEM) MODEL" # @param {type:"string"}
#@markdown * Путь к выходной папке:
output_dir = "/content/output" # @param {type:"string"}
# @markdown * Показать список выходных файлов
show_list_outputs = True # @param {"type":"boolean"}

%cd /content/mvsepless-gamma

cmd = [
    "python multi_inference.py separate",
    f"-i \"{input_path}\"",
    f"-o \"{output_dir}\"",
    f"-mt \"{model_type}\"",
    f"-mn \"{model_name}\"",
    f"-of \"{output_format}\"",
    f"-bitrate \"{bitrate}k\"",
    f"-vr_aggr {vr_arch_aggressive}",
    f"--template \"{output_template}\""
]

if instrumental:
    cmd.append("--instrumental")

if show_list_outputs:
    cmd.append("-l_out")

if stems_to_extract:
    cmd.append(f"--stems {stems_to_extract}")

!{" ".join(cmd)}

In [32]:
#@title Показать список моделей
#@markdown * Фильтр по стему
filter = "" # @param {"type":"string"}
%cd /content/mvsepless-gamma
if filter or filter != "":
    filter = f"-l_filter '{filter}'"
else:
    filter = ""


cmd = f"python multi_inference.py list {filter}"
!$cmd

/content/mvsepless-gamma
+-------------------+---------------------------------------------------------+-------------------------------------------+---------------------+----------------+
| Model Type        | Model Name                                              | Stems                                     | Target Instrument   | Primary Stem   |
+===================+=========================================================+===========================================+=====================+================+
| mel_band_roformer | Mel-Band-Roformer_Vocals_kimberley_jensen               | vocals, other                             | vocals              | N/A            |
+-------------------+---------------------------------------------------------+-------------------------------------------+---------------------+----------------+
| mel_band_roformer | Mel-Band-Roformer_InstVoc_Duality_v1_unwa               | Vocals, Instrumental                      |                     | N/A           